## 06 Decision trees and ensembling

*special thanks to YSDA team for provided materials*`

![img](https://pbs.twimg.com/media/B13n2VVCIAA0hJS.jpg)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Let's generate a toy dataset:

In [ ]:
from sklearn.datasets import make_blobs

X_toy, y_toy = make_blobs(n_samples=400,
                          centers=[[0., 1.], [1., 2.]],
                          random_state=14)

plt.scatter(X_toy[:, 0], X_toy[:, 1], c=y_toy, alpha=0.8, cmap='bwr')
plt.xlabel('X1'), plt.ylabel('X2');

## Decision trees out of the box

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

DecisionTreeClassifier has a number of parameters:
* `max_depth` – a limit on tree depth (default – no limit)
* `min_samples_split` – there should be at least this many samples to split further (default – 2)
* `min_samples_leaf` – there should be at least this many samples on one side of a split to consider it valid (default – 1).
* `criterion` – 'gini' or 'entropy' – split stuff over this parameter (default : gini)

In [ ]:
clf = DecisionTreeClassifier(min_samples_leaf=15)
clf.fit(X_toy, y_toy)

### Plot decision surface

Here's a function that makes a 2d decision boundary plot for a given classifier:

In [ ]:
from sklearn.metrics import accuracy_score

def plot_decision_surface(
                  clf, X, y,
                  grid_step=0.02,
                  cmap='bwr',
                  alpha=0.6,
        ):
    """
    Plot the decision boundary of clf on X and y, visualize training points
    """
    
    # Define the grid
    x_top_left = X.min(axis=0) - 1
    x_bottom_right = X.max(axis=0) + 1
    grid_x0, grid_x1 = np.meshgrid(
         np.arange(x_top_left[0], x_bottom_right[0], grid_step),
         np.arange(x_top_left[1], x_bottom_right[1], grid_step)
      )
    
    # Calculate predictions on the grid
    y_pred_grid = clf.predict(
                        np.stack(
                              [
                                grid_x0.ravel(),
                                grid_x1.ravel()
                              ],
                              axis=1
                            )
                      ).reshape(grid_x1.shape)
    
    # Find optimal contour levels and make a filled
    # contour plot of predictions
    labels = np.sort(np.unique(y))
    labels = np.concatenate([[labels[0] - 1],
                             labels,
                             [labels[-1] + 1]])
    medians = (labels[1:] + labels[:-1]) / 2
    plt.contourf(grid_x0, grid_x1, y_pred_grid, cmap=cmap, alpha=alpha,
                 levels=medians)
    
    # Scatter data points on top of the plot,
    # with different styles for correct and wrong
    # predictions
    y_pred = clf.predict(X)
    plt.scatter(*X[y_pred==y].T, c=y[y_pred==y],
                marker='o', cmap=cmap, s=10, label='correct')
    plt.scatter(*X[y_pred!=y].T, c=y[y_pred!=y],
                marker='x', cmap=cmap, s=50, label='errors')

    # Dummy plot call to print the accuracy in the legend.
    plt.plot([], [], ' ',
             label='Accuracy = {:.3f}'.format(accuracy_score(y, y_pred)))
    
    plt.legend(loc='best')

Let's apply it to the tree we've fitted above:

In [ ]:
plt.figure(figsize=(12, 8))
plot_decision_surface(clf, X_toy, y_toy)

### Tree depth

First we are going to split our data to train and test subsets:

In [ ]:
X_toy_train, X_toy_test, y_toy_train, y_toy_test = \
    train_test_split(X_toy, y_toy, test_size=0.25)

Now it's your turn to investigate how the decision boundary depends on the tree depth. Maximum tree depth is defined by the `max_depth` parameter. Try out the following values: `[1, 2, 3, 5, 10]`. Make decision boundary plots for both train and test datasets (separately).

  > *Hint: you can make a nice plot with multiple columns and rows (see example below).*
  
```python
plt.figure(figsize=(width, height))
for i in range(num_rows * num_columns):
  plt.subplot(num_rows, num_columns, i + 1)
  # subplot numbering starts from 1   ^^^
  
  # ...
  # (do the plotting)
plt.show();
```

In [ ]:
depth_values = [1, 2, 3, 5, 10]

num_rows, num_columns = # <YOUR CODE HERE>

plt.figure(figsize=(15,25))
for i in range(num_rows):
    # <YOUR CODE HERE>
    plt.subplot(num_rows, num_columns, 2 * i + 1)
    # <YOUR CODE HERE>
    # subplot numbering starts from 1   ^^^
plt.show();

### Toy multiclass data

Now let's try out a multiclass classification case:

In [ ]:
!wget https://github.com/yandexdataschool/MLatImperial2020/raw/master/04_lab/data.npz

Firstly, we'll load the data:

In [ ]:
data = np.load('data.npz')
X, y = data["X"], data["y"]

print(X.shape, y.shape)

And then split it to train and test:

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.5, random_state=1337)

Now it's your turn to have a look at the data. Make a 2d scatter plot of the data points.

 > *Hint: instead of calling `scatter` separately for each class, you can give it a vector of color index values through the `c` parameter (`scatter(x0, x1, c=y, cmap='rainbow'`). The 'rainbow' colormap gives good enough color diversity for the multiclass case.*

In [ ]:
plt.figure(figsize=(10, 7))
#<YOUR CODE HERE>

Now that we've had a look at the data, let's fit a decision tree on it:

In [ ]:
# YOUR CODE HERE

and plot the result:

In [ ]:
plt.figure(figsize=(12, 16))
plt.subplot(2, 1, 1)
plot_decision_surface(clf, X_train, y_train, cmap='rainbow', grid_step=0.2)
plt.subplot(2, 1, 2)
plot_decision_surface(clf, X_test, y_test, cmap='rainbow', grid_step=0.2);

```

```

```

```

```

```

```

```

#### We need a better tree!

Try adjusting the parameters of DecisionTreeClassifier to improve the test accuracy.
 * Accuracy >= 0.72 - not bad for a start
 * Accuracy >= 0.75 - better, but not enough
 * Accuracy >= 0.77 - pretty good
 * Accuracy >= 0.78 - great! (probably the best result for a single tree)
 
Feel free to modify the DecisionTreeClassifier above instead of re-writing everything.

**Note:** some of the parameters you can tune are under the "Decision trees out of the box" header.

#### Bonus quest
Try adding feature transformations using a pipeline and a function transformer, e.g.:
```python
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

clf = make_pipeline(
    FunctionTransformer(lambda X: np.concatenate([X, X**2], axis=1)),
    DecisionTreeClassifier()
)
```

Which transformations should improve the score?

```
```
```
```

We've talked a lot about the importance of feature scaling. Why aren't we doing it here?

Try adding a standard scaler to the pipeline of your best model and check how it affects the result. Can you explain the result?

In [ ]:
# <YOUR CODE>

```
```
```
```

## Ensembles

### Optional quest: AdaBoost from scratch

**Implement `Adaboost` class** (for classification taks)

**Details:**
- inherit class from `sklearn.BaseEstimator`;
- constructor must support the following parameters: 
    - `n_estimators` - the number of base models (decision stomps) for the ensemble;
    - `max_depth` - max depth for each decision tree (default - `None`);
- methods `fit` и `predict` must be implemented;
- method `fit` takes design matrix `X` and target vector `y` as inputs (`numpy.ndarray`) and returns `Adaboost` class, which is an ensemble of base estimators, trained using a dataset `(X, y)` given the parameters, passed to the constructor; 

In [ ]:
from sklearn.base import BaseEstimator
class Adaboost(BaseEstimator):
    """
    Boosting method that uses a number of weak classifiers in 
    ensemble to make a strong classifier. This implementation uses DecisionTreeClassifier. 
    
    Parameters:
    -----------
    n_clf: int
        The number of weak classifiers that will be used. 
    max_depth: int
        Maximal depth of the decision tree inside the ensemble
    """
    def __init__(self, n_estimators=5, max_depth=1, random_state=0):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.alphas = []
        self.clfs = []
        self.random_state = random_state
        
    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Initialize all weights as 1/N
        w = np.full(n_samples, (1 / n_samples))
        
        self.clfs = []
        self.alphas = []
        
        _eps = 1e-10
        # Iterative fitting of the "weak" base estimators
        for _ in range(self.n_estimators):
            
            clf = <YOUR CODE HERE>
            
            min_error = <YOUR CODE HERE>
            
            # Compute `alpha` parameter, which is used to update the weights of the samples from the training set
            # `alpha` may be considered as the  algorithm "performance" level approximation
            alpha = <YOUR CODE HERE>
            
            # Update the weights of the training set samples
            # Misclassified samples get larger weights, than those with correct class predictions
            w *= <YOUR CODE HERE>
            
            # Updated weights normalization
            w /= np.sum(w)
            
            # Save the base estimator and add its `alpha` parameter to the corresponding list
            self.clfs.append(clf)
            self.alphas.append(alpha)

    def predict(self, X):
        n_samples = np.shape(X)[0]
        y_pred = np.zeros((n_samples, 1))
        
        # Get the prediction from the every model, taking into account the corresponding `alpha` value
        for alpha, clf in zip(self.alphas, self.clfs):
            # Sum up the predictions weighted with the corresponding `alpha` values
            y_pred += <YOUR CODE HERE>

        # Class prediction is defined as the sign of weighted sum of the base estimator predictions
        y_pred = np.sign(y_pred).flatten()

        return y_pred

Load the toy dataset to evaluate the ensemble performance

In [ ]:
from sklearn.datasets import load_digits

data = load_digits()
X = data.data
y = data.target

# select two different digits to construct the binary classification problem
digit1 = 1
digit2 = 8
idx = np.append(np.where(y == digit1)[0], np.where(y == digit2)[0])
y = data.target[idx]

# Change class labels to -1 and 1
y[y == digit1] = -1
y[y == digit2] = 1
X = data.data[idx]

# split the dataset into training and testing sets
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.5, random_state=42)

Set up AdaBoost hyperparameters

In [ ]:
# set base estimator number
n_estimators = 12
max_depth = 1

# set up base estimator
base_clf = DecisionTreeClassifier(random_state=42, max_depth=max_depth)
base_clf.fit(train_X, train_Y)

# save base estimator performance for future comparison
y_pred_base = base_clf.predict(test_X)
print ("Decision Stump accuracy: {:.5f}".format(accuracy_score(test_Y, y_pred_base)))

Let's create a reference model using scikit-learn version of AdaBoostClassifier:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# use scikit-learn version as a reference
ada = AdaBoostClassifier(
    base_estimator=base_clf,
    n_estimators=n_estimators,
    random_state=42,
    algorithm='SAMME')

ada.fit(train_X, train_Y)
y_pred_ada = ada.predict(test_X)

sklearn_accuracy = accuracy_score(test_Y, y_pred_ada)
print ("Scikit-learn' AdaBoost with {} estimators accuracy: {:.5f}".format(n_estimators, sklearn_accuracy))

And here we are finally using our own class:

In [ ]:
clf = Adaboost(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
clf.fit(train_X, train_Y)

y_pred = clf.predict(test_X)
my_accuracy = accuracy_score(test_Y, y_pred)

assert np.allclose(my_accuracy, sklearn_accuracy), 'Your implementation accuracy does not match the sklearn version'
print ("My AdaBoost implementation with {} estimators accuracy: {:.5f}".format(n_estimators, my_accuracy))

```
```
```
```

### Pre-implemented ensembles: Gradient Boosting

One of the most commonly used libraries for gradient boosing is the [XGBoost library](https://xgboost.ai/). Consider reading [this document](https://xgboost.readthedocs.io/en/latest/tutorials/model.html) for an introduction to the algorithm.

Here's the [help page](https://xgboost.readthedocs.io/en/latest/parameter.html) listing available parameters.

Let's start by importing the classifier class and the function that plots individual trees as graphs:

In [ ]:
from xgboost import XGBClassifier, plot_tree

We can now investigate how decision surface depends on the number of trees:

In [ ]:
for n_estimators in range(1,10):
    # create an XGBClassifier with trees of depth 1
    # learning rate 0.5 and n_estimators estimators
    model = <YOUR CODE HERE> 
    
    # fit this model to the train data
    <YOUR CODE HERE> 
    
    print("n_estimators = ", n_estimators)
    plt.figure(figsize=(8, 5))
    plot_decision_surface(model, X_test, y_test, cmap='rainbow', grid_step=0.4)
    plt.show()

And here's how one may use the `plot_tree` function:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
plot_tree(model, num_trees=44, ax=ax, dpi='400');
#                   ^^^ This parameter selects the
#                       tree that you want to plot.
#                       Since there's 9 estimators
#                       in the last model and 5
#                       classes in our data, the total
#                       amount of individual trees
#                       is 45 (from 0 to 44).

<font color='red'>**Warning:**</font> current xgboost implementation is not very safe to typos, i.e. it can silently swallow whatever argument you provide, even if it has no effect, e.g.:

In [ ]:
model = XGBClassifier(abrakadabra="I won't change anything")

so be sure to check your spelling.

Now let's try to improve the score by adjusting the parameters. Here are some of the parameters you may want to try:
  - `max_depth` – maximum tree depth,
  - `n_estimators` – number of trees (per class),
  - `learning_rate` – shrinkage,
  - `reg_lambda` – L2 regularization term on weights,
  - `subsample` – row random subsampling rate (per tree),
  - `colsample_bynode` – column subsampling rate (per node)
  - `gamma` – minimum loss reduction required to make a further partition on a leaf node of the tree

See [this page](https://xgboost.readthedocs.io/en/latest/parameter.html) for more information.

  > *Hint: since XGBClassifier has the same interface as sklearn models, you can use GridSearchCV on it if you want.* 

In [ ]:
model = <YOUR CODE HERE> 
model.fit(X_train, y_train)

plt.figure(figsize=(8, 5))
plot_decision_surface(model, X_test, y_test, cmap='rainbow', grid_step=0.8)
plt.show()

```
```
```
```

### Pre-implemented ensembles: Random Forest

RandomForest combines bagging and random subspaces: each tree uses a fraction of training samples, and the splits are chosen among subsets of features. Typically this leads to a slightly better performance.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Task: create and fit a random forest with
# 100 estimators and at least 5 samples per leaf

model = <YOUR CODE>

<YOUR CODE>

plt.figure(figsize=(12, 8))
plot_decision_surface(model, X_test, y_test, cmap='rainbow', grid_step=0.2)

## Bonus part: visualizing a tree

In [ ]:
import pydotplus
import io
import os

from sklearn.tree import plot_tree, export_graphviz
from IPython.display import SVG

def plot_tree_svg(tree, columns, max_depth):
    buf = io.StringIO()
    export_graphviz(tree, out_file=buf,
                  feature_names=columns,
                  max_depth=max_depth)
    graph = pydotplus.graph_from_dot_data(buf.getvalue())
    return SVG(graph.create_svg())

# there are complications running this on windows, so there is a workaround

if os.name == 'nt':
    plot_tree(clf.clfs[0], feature_names=list(range(64)), max_depth=3)
else:
    plot_tree_svg(clf.clfs[0], list(range(64)), max_depth=3)

```
```
```
```